In [101]:
import numpy as np
import time

### Define vehicles and tasks sets

In [102]:
vehicles = [0,1,2,3]
tasks = [0,1,2]

### Create random utility matrix

We use a matrix to store the utilities of the vehicles for the different possible allocations.
The advantage of using a matrix compared to storing all the possible combinations in a dictionary, a hashmap or similar, is that we save a lot of space because we don't have to store the keys, and the acces time is the same. The other advantage is that we can loop easily on the matrix and use optimized numpy functions over this matrix. Using a matrix could also be interesting to move the computations on GPU to speed it up for high dimentionnal instances.

In [3]:
# Clean version :

def getShape(nb_vehicles, nb_tasks):
    """ Compute the shape for this settings
    
    Parameters :
        nb_vehicles : int, the number of vehicles
        nb_tasks : int, the number of tasks
        
    Returns :
        a tuple containing the shape (tasks^vehicles * vehicles),
        that is the shape for the utility matrix.
        
    """
    list_dim = [nb_tasks]*nb_vehicles +[nb_vehicles]# matrix of shape tasks^vehicles * vehicles
    return tuple(list_dim)

shapes = getShape(len(vehicles),len(tasks))
shapes

(3, 3, 3, 3, 4)

In [137]:
ut = np.random.randint(0,10,shapes) # create utility matrix with random utilities

In [155]:
ut[0][0][0][0] # get utility of all agents when all do task 0

array([6, 0, 9, 4])

In [156]:
ind = tuple([0,0,0,0]) # accessing the matrix from array as index
ut[ind]

array([6, 0, 9, 4])

In [157]:
ut[0][1][2][2] # get utility of all agents when allocation for vehicles to task is = 0,1,2,2 
# (means vehicule 0 do task 0, vehicule 1 do task 1, vehicule 2 do task 2, vehicule 3 do task 2)

array([9, 9, 5, 8])

### Defining usefull functions

In [21]:
def replaceAlloc(allocation, v, t):
    """ Compute the new allocation with task t asigned to vehicle v
    
    Parameters :
        allocation : List(int) the list of tasks allocades to each vehicle (in order)
        v : int, the vehicle id
        t : int, the task id
    
    Returns :
        List(int), the new allocation
    
    """
    return list(allocation[:v])+[t]+list(allocation[v+1:])

In [5]:
def is_EN(utilities, allocation, vehicles, tasks):
    """ Check if the current allocation is a Nash Equilibrium or not
    
    Parameters :
        utilities : Matrix(int) the utility matrix of dimension nb_tasks^nb_vehicles * nb_vehicles
        allocation : List(int) the list of tasks allocades to each vehicle (in order)
        vehicle : List(int) the list of vehicle ids
        tasks : List(int) the list of task ids
    
    Returns :
        Tuple(boolean, int)
        A tuple containing a boolean (True if this allocation is a Nash Equilibrium, else False)
        and an integer that is the id of a vehicle that can increase its utility
        by changing unilateraly its allocation (if not EN, -1)
    
    """
    for v in range(len(vehicles)) : # for each vehicle
        current_task = allocation[v]
        current_utility = utilities[tuple(allocation)][v]
        for t in range(len(tasks)) :
            if t != current_task : # check all other tasks
                temp_ind = replaceAlloc(allocation, v, t) # allocating task t to vehicle v
                utility = utilities[tuple(temp_ind)][v]
                if utility > current_utility : # changing to another task gives more utility -> Not NE
                    return (False, v)
    return (True, -1)

We also return the id of one vehicle that can increase its utility by changing its allocation, if the solution is not a Nash Equilibrium. 
It didn't increase the computation cost and avoids looping another time later on the utility table to find one in the Best Response Dynamics, it's all benefits.

##### Nash Equilibrium test example on small dimension

In [224]:
# Create setup : 2 vehicles, 3 tasks
v = [0,1] # don't change, it's 2D example
t = [0,1,2]
ut_test = np.random.randint(0,10,(len(t),len(t),2))
ut_test

array([[[0, 4],
        [3, 7],
        [7, 2]],

       [[9, 2],
        [2, 0],
        [8, 8]],

       [[8, 2],
        [2, 8],
        [9, 1]]])

In [225]:
# Check EN for allocation (0,0)
alloc = [0,0] # set allocation to check (0,1) -> first vehicle do task 0 and second do task 1
is_EN(ut_test, alloc, v, t) # random example

(False, 0)

In [226]:
alloc = [0,1]
ut_test[tuple(alloc)] = [0,0] # set allocation (0,1) to the lowest value for each vehicle
ut_test[2,1] = [1,0] # set another allocation to a better score for one vehicle (in case matrix is full zero)
is_EN(ut_test,alloc , v, t) # -> Is necessarly not an EN (result must be False)

(False, 0)

In [227]:
alloc = [0,1]
ut_test[tuple(alloc)] = [10,10] # set allocation (0,1) to the highest value for each vehicle
is_EN(ut_test, alloc, v, t) # -> Is necessarly  an EN (result must be True)

(True, -1)

In [228]:
ut_test_z = np.zeros((len(t),len(t),2)) # set all matrix to 0 (same value everywhere)
ut_test_z

array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]])

In [229]:
alloc = [2,1] # when Zero everywhere, no solution is strictly better than the current
is_EN(ut_test_z, alloc, v, t) # -> Is necessarly an EN (result must be True)

(True, -1)

### Best Response Dynamics

In [6]:
def getBestTask(utilities, allocation, v, tasks):
    """ Compute the best task for vehicle v
    
    Parameters :
        utilities : Matrix(int) the utility matrix of dimension nb_tasks^nb_vehicles * nb_vehicles
        allocation : List(int) the list of tasks allocades to each vehicle (in order)
        v : int, the vehicle id
        tasks : List(int) the list of task ids
        
    Returns : 
        int, the best task for vehicle v
    """
    best = np.argmax([utilities[tuple(replaceAlloc(allocation, v, t))][v] for t in range(len(tasks))])
    return best

In [7]:
def bestResponseDynamic(utilities, vehicles, tasks, maxsteps):
    """ Try to compute a Nash Equilibrium allofaction using Best Response Dynamics
    
    Parameters :
        utilities : Matrix(int) the utility matrix of dimension nb_tasks^nb_vehicles * nb_vehicles
        vehicles : List(int), the list of vehicle ids
        tasks : List(int) the list of task ids
        maxteps : int, the steps limit of the algorithm
        
    Returns : 
        List(int), a Nash Equilibrium allocation if one was found (no guarantee)
        
    """
    allocation = np.random.randint(0,len(tasks),len(vehicles)) # initial random allocation
    end, id_change = is_EN(utilities, allocation, vehicles, tasks)
    steps = 0
    while not(end) and steps < maxsteps:
        # vehicle id_change has interest to change to a better allocation
        best = getBestTask(utilities, allocation, id_change, tasks) # get its best unilateral allocation
        allocation = replaceAlloc(allocation, id_change, best) # set next allocation for id_change
        end, id_change = is_EN(utilities, allocation, vehicles, tasks)
        steps += 1
    if not(end) and steps >= maxsteps: # cut the exection if maxsteps reached
        print("Execution stopped : maximum step overflowed, no EN found.")
    return list(allocation)

##### Best Response Dynamics test example on the initial matrix

In [22]:
ut[2,2,1,1] = [10,10,10,10] # we create a global optimal affectation -> At least one EN in utilities table

In [292]:
bestResponseDynamic(ut, vehicles, tasks, 1000) # the global optimum is reached (NE)

[2, 2, 1, 1]

In [291]:
bestResponseDynamic(ut, vehicles, tasks, 1000) # another Nash Equilibrium is sometimes reached

[0, 2, 0, 2]

### Fictitious Play

In [10]:
shapes = getShape(len(vehicles),len(tasks))
print(shapes)
ut = np.random.randint(0,10,shapes) # create utility matrix with random utilities

(3, 3, 3, 3, 4)


In [8]:
def computeFrequency(proposals, tasks):
    """ Compute the empirical frequency of each proposal
    
    Parameters :
        proposals : List(int), the allocation proposed
        tasks : List(int) the list of task ids
        
    Returns : 
        List(float), the empirical frequency of each proposal
    
    """
    return [np.count_nonzero(np.array(proposals) == t)/len(proposals) for t in range(len(tasks))]

In [10]:
# testing frequencies computation
proposals = [0,1,2,1,1,2,0,0,1,2,2,1,2,0,1,1,2,1,2,0,1] # proposals sequence made by an agent
taks = [0,1,2] # define tasks

In [11]:
computeFrequency(proposals,tasks) # compute frequency of each task in proposals

[0.23809523809523808, 0.42857142857142855, 0.3333333333333333]

In [9]:
def computeFullFrequencyMatrix(frequencies, vehicles, tasks): # not used
    """
        Compute the joined frequency matrix of proposed allocation for each vehicle
    """
    shapes = getShape(len(vehicles),len(tasks))
    fMat = np.zeros(shapes)
    allAlloc = [x for x,_ in np.ndenumerate(ut)]
    for alloc in allAlloc :
        fMat[alloc] = np.prod([frequencies[v][alloc[v]] for v in range(len(vehicles))])
    return fMat

In [36]:
# testing frequency matrix computation
frequencies = np.random.random((4,10)) # random frequencies

In [37]:
frequencies

array([[0.43819445, 0.02242756, 0.75844368, 0.34672042, 0.84612548,
        0.63451129, 0.98444621, 0.92280188, 0.96361494, 0.62077178],
       [0.34867839, 0.66745858, 0.15330373, 0.41385088, 0.41217432,
        0.93827241, 0.81652892, 0.39361675, 0.39504645, 0.81168782],
       [0.09905056, 0.30935464, 0.63764402, 0.28090354, 0.04137854,
        0.59448294, 0.41146702, 0.76507212, 0.61917395, 0.35979492],
       [0.99832658, 0.62650982, 0.44820006, 0.44499181, 0.29123082,
        0.08235175, 0.8698987 , 0.12316711, 0.70671548, 0.07297071]])

In [38]:
computeFullFrequencyMatrix(frequencies, vehicles, tasks)

array([[[[[1.51085040e-02, 1.51085040e-02, 1.51085040e-02,
           1.51085040e-02],
          [9.48149258e-03, 9.48149258e-03, 9.48149258e-03,
           9.48149258e-03],
          [6.78298311e-03, 6.78298311e-03, 6.78298311e-03,
           6.78298311e-03]],

         [[4.71868703e-02, 4.71868703e-02, 4.71868703e-02,
           4.71868703e-02],
          [2.96125917e-02, 2.96125917e-02, 2.96125917e-02,
           2.96125917e-02],
          [2.11846086e-02, 2.11846086e-02, 2.11846086e-02,
           2.11846086e-02]],

         [[9.72619179e-02, 9.72619179e-02, 9.72619179e-02,
           9.72619179e-02],
          [6.10376879e-02, 6.10376879e-02, 6.10376879e-02,
           6.10376879e-02],
          [4.36658683e-02, 4.36658683e-02, 4.36658683e-02,
           4.36658683e-02]]],


        [[[2.89214956e-02, 2.89214956e-02, 2.89214956e-02,
           2.89214956e-02],
          [1.81499734e-02, 1.81499734e-02, 1.81499734e-02,
           1.81499734e-02],
          [1.29843442e-02, 1.298434

In [20]:
def computePartialFrequencyMatrix(utilities, frequencies, vehicles, tasks, v):
    """
        Compute the expected utility for vehicle v foreach task, considering for each 
        other vehicle a random allocation choice with probability equal to empirical frequency observed 
        
    """
    temp_vehicles = vehicles[:v] + vehicles[v+1:] # create a list of index without vehicle v
    temp_alloc = np.zeros([len(tasks)]*(len(vehicles)-1)) # create a 0 array of shape (vehicles-1)^task
    allAlloc = [x for x,_ in np.ndenumerate(temp_alloc)] # enumerate all possible allocations for these vehicles
    expectations = []
    for t in range(len(tasks)):
        expected = 0
        for alloc in allAlloc : # for each possible allocation for vehicles without v
            # compute proba for these vehicles to do this allocation : 
            proba = np.prod([frequencies[temp_vehicles[v]][alloc[v]] for v in range(len(temp_vehicles))])
            # recreating full allocation with v :
            index = [0]*len(vehicles)
            for i in range(len(temp_vehicles)) :
                index[temp_vehicles[i]] = alloc[i]
            index[v] = t
            # get the utility for v if it do task t with this allocation for the other vehicles
            utility = utilities[tuple(index)][v]
            expected+= utility * proba # add proba time the utility of v to the expectation
        expectations.append(expected)
    return expectations

In [12]:
# testing expectation computation
frequencies = np.random.random((4,10)) # random frequencies (note that sum != 1, it's just for example)
frequencies

array([[0.334678  , 0.3732251 , 0.04527724, 0.48045606, 0.63339386,
        0.29691106, 0.11301805, 0.4852836 , 0.5705335 , 0.04129699],
       [0.97844816, 0.88709069, 0.92355253, 0.8601529 , 0.60998929,
        0.96940864, 0.30773177, 0.49419312, 0.19695949, 0.56050583],
       [0.28566915, 0.56251782, 0.85265831, 0.20938456, 0.43540033,
        0.52447368, 0.46527975, 0.31814684, 0.72949375, 0.09634318],
       [0.64480135, 0.88624043, 0.4386029 , 0.76159956, 0.16887837,
        0.19771879, 0.60571374, 0.79471096, 0.04598445, 0.68379359]])

In [76]:
for v in vehicles : # get the task with the highest utility expectation for each vehicle
    expect = computePartialFrequencyMatrix(ut, frequencies, vehicles, tasks, v)
    print(str(expect)+" --> task "+str(np.argmax(expect)))

[19.52697896567259, 18.730815737618343, 24.867858909366248] --> task 2
[16.5332330478646, 20.729079619506614, 16.17518652391417] --> task 1
[12.996964693052485, 14.300402422881186, 12.27633887531842] --> task 1
[10.923915888343553, 11.52656292187575, 7.27074793265533] --> task 1


In [19]:
def fictitiousPlay(utilities, vehicles, tasks, maxsteps):
    """ Play Fictitious Play until maxsteps
    
    Parameters :
        utilities : Matrix(int) the utility matrix of dimension nb_tasks^nb_vehicles * nb_vehicles
        vehicles : List(int), the list of vehicle ids
        tasks : List(int) the list of task ids
        maxteps : int, the steps limit of the algorithm
        
    Returns : 
        List(int), the found allocation after maxsteps steps (no optimality guarantee)
        
    """
    passedPropositions = [[t for t in tasks] for v in vehicles]
    steps = 0
    while steps < maxsteps:
        for v in vehicles :
            f = [computeFrequency(passedPropositions[v],tasks) for v in vehicles]
            expect = computePartialFrequencyMatrix(utilities, f, vehicles, tasks, v)
            bestTask = np.argmax(expect)
            passedPropositions[v].append(bestTask)
        #print([passedPropositions[v][-1] for v in vehicles])
        steps += 1
    return list([passedPropositions[v][-1] for v in vehicles])

In [213]:
# testing FP compared to BRD on some examples :

In [204]:
bestResponseDynamic(ut, vehicles, tasks, 1000)

[0, 0, 2, 1]

In [34]:
bestResponseDynamic(ut, vehicles, tasks, 1000)

[2, 2, 0, 1]

In [36]:
bestResponseDynamic(ut, vehicles, tasks, 1000)

[1, 1, 1, 2]

In [207]:
fictitiousPlay(ut, vehicles, tasks, 1000)

[2, 1, 2, 0]

In [212]:
fictitiousPlay(ut, vehicles, tasks, 1000) # this is a better solution than BRD's NE in term of social utility

[2, 1, 1, 0]

In [58]:
# (if cells before have been re-run, it's no more the same ut matrix so no more the same results)
# obtained with fictitious play :  
# [2,1,1,0]  (maxsteps 500)
# [2,1,2,0]  (maxsteps 1500)
print("FP : "+str(ut[2,1,1,0])+" -> mean "+str(np.mean(ut[2,1,1,0])))
print("FP : "+str(ut[2,1,2,0])+" -> mean "+str(np.mean(ut[2,1,2,0])))

# obtained with best response dynamic : 
# [1,2,0,0]  (maxsteps 1000)
# [2,0,1,2]  (maxsteps 1000)
# [0,0,2,1]  (maxsteps 1000)

print("BRD : "+str(ut[1,2,0,0])+" -> mean "+str(np.mean(ut[1,2,0,0])))
print("BRD : "+str(ut[2,0,1,2])+" -> mean "+str(np.mean(ut[2,0,1,2])))
print("BRD : "+str(ut[0,0,2,1])+" -> mean "+str(np.mean(ut[0,0,2,1])))

FP : [5 1 6 1] -> mean 3.25
FP : [2 2 1 7] -> mean 3.0
BRD : [9 6 7 2] -> mean 6.0
BRD : [1 2 8 7] -> mean 4.5
BRD : [3 1 5 8] -> mean 4.25


### Regret Matching

In [18]:
def computeR(R, u, k, tasks, alloc, v): # not used
    """ Update the average regret table of vehicle v for not proposing each task
    
    Parameters:
        R : np.array(float) the regret table for each action at each step k (size: steps*tasks)
        u : the utility matrix
        k : int, the current time step
        tasks : List(int), the list of tasks id
        alloc : List(int), the current allocation
        v : int, the vehicle
    
    """
    if k == 0:
        return
    for t in tasks :
        loss = u[tuple(replaceAlloc(alloc, v, t))][v] - u[alloc][v]
        R[k+1,t] = ((k-1)/k)*R[k,t] + (1/k) * loss

def getNextR(lastR, u, k, tasks, alloc, v):
    """ Compute the average regret vector of vehicle v for not proposing each task, based on last regret vector
    
    Parameters:
        lastR : np.array(float) the regret vactor for each task at last step k-1
        u : the utility matrix
        k : int, the current time step
        tasks : List(int), the list of tasks id
        alloc : List(int), the current allocation
        v : int, the vehicle
    Return:
        np.array(float) the regret vactor for each action at current step k
    """
    if k == 0:
        return np.zeros(len(tasks))
    else :
        return np.array([((k-1)/k)*lastR[t] + (1/k) * (u[tuple(replaceAlloc(alloc, v, t))][v] - u[alloc][v]) for t in tasks])
        

In [50]:
# computing regret during random play using a regret table
v=0
total_steps = 10
R = np.zeros((total_steps, len(tasks)))
for i in range(9) : # computing regret during random play
    computeR(R, ut, i, tasks, tuple(np.random.randint(0,len(tasks),len(vehicles))),v)

In [51]:
R # regret at each step k

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [-7.        ,  0.        ,  1.        ],
       [-4.5       , -2.5       ,  0.5       ],
       [-2.        , -1.66666667,  1.66666667],
       [-0.5       , -1.25      ,  0.5       ],
       [-1.8       , -1.        ,  0.4       ],
       [-2.33333333, -0.83333333, -0.16666667],
       [-2.        ,  0.57142857,  0.85714286],
       [-1.25      ,  0.5       ,  0.25      ]])

In [16]:
# computing it during random play with last regret only (no regret table)
v=0
lastR = np.zeros(len(tasks)) 
for k in range(9) : # computing regret during random play
    lastR = getNextR(lastR, ut, k, tasks, tuple(np.random.randint(0,len(tasks),len(vehicles))),v)
    print(lastR)

[0. 0. 0.]
[6. 0. 5.]
[6. 0. 5.]
[4.33333333 0.         5.66666667]
[4.75 0.   5.5 ]
[3.8 0.  4.6]
[3.5 0.  3.5]
[2.71428571 0.         3.        ]
[2.5   0.375 2.625]


In [17]:
def probaFromRegret(R):
    """ Compute the probability to propose each task depending on the regret vector
    
    Parameters:
        R : np.array(float) the regret vector for each action at current step k     
    Return:
        np.array(float) the probability vector for each action at current step k
    
    """
    Rn = np.array(R)
    positive_reg = np.where(Rn > 0, Rn, 0)
    if np.sum(positive_reg) == 0:
        proba = np.ones(len(R))/len(R)
    else :
        proba = positive_reg/np.sum(positive_reg)
    return proba

In [17]:
print("regret vector for v : "+str(lastR))
print("proba : ")
print(probaFromRegret(lastR)) # getting proba based on lastR regret vector
print("proba when all regrets <=0 : ")
print(probaFromRegret([0,0,0])) # if all <=0 : same proba for all

regret vector for v : [2.5   0.375 2.625]
proba : 
[0.45454545 0.06818182 0.47727273]
proba when all regrets <=0 : 
[0.33333333 0.33333333 0.33333333]


In [16]:
def RegretMatching(utilities, vehicles, tasks, maxsteps):
    """ Play Regret Matching until maxsteps
    
    Parameters :
        utilities : Matrix(int) the utility matrix of dimension nb_tasks^nb_vehicles * nb_vehicles
        vehicles : List(int), the list of vehicle ids
        tasks : List(int) the list of task ids
        maxteps : int, the steps limit of the algorithm
        
    Returns : 
        List(int), the found allocation after maxsteps steps (no optimality guarantee)
        
    """
    k = 0
    lastR = np.zeros((len(vehicles), len(tasks)))
    cur_alloc = np.random.randint(0,len(tasks), len(vehicles))
    next_alloc = np.random.randint(0,len(tasks), len(vehicles))
    while k < maxsteps:
        for v in vehicles:
            lastRv = lastR[v]
            nextRv = getNextR(lastRv, ut, k, tasks, tuple(cur_alloc), v)
            proba = probaFromRegret(nextRv)
            task = np.random.choice(tasks, p=proba)
            next_alloc[v] = task
            lastR[v] = nextRv # update the regret of v
        #print(next_alloc)
        cur_alloc = next_alloc # update the allocation
        k += 1
    return list(cur_alloc)

In [95]:
# some runs of the 3 methods on the same ut matrix :

In [19]:
RegretMatching(ut, vehicles, tasks, 1000) 

[2, 1, 2, 1]

In [22]:
RegretMatching(ut, vehicles, tasks, 1000)

[1, 1, 0, 1]

In [21]:
RegretMatching(ut, vehicles, tasks, 1000)

[0, 1, 2, 1]

In [23]:
fictitiousPlay(ut, vehicles, tasks, 1000)

[1, 1, 2, 0]

In [24]:
fictitiousPlay(ut, vehicles, tasks, 1000)

[1, 1, 2, 0]

In [25]:
fictitiousPlay(ut, vehicles, tasks, 1000)

[1, 1, 2, 0]

In [26]:
bestResponseDynamic(ut, vehicles, tasks, 1000)

Execution stopped : maximum step overflowed, no EN found.


[0, 1, 1, 2]

In [35]:
bestResponseDynamic(ut, vehicles, tasks, 1000)

Execution stopped : maximum step overflowed, no EN found.


[1, 2, 1, 2]

In [28]:
bestResponseDynamic(ut, vehicles, tasks, 1000)

Execution stopped : maximum step overflowed, no EN found.


[0, 1, 1, 2]

### Spatial Adaptative Play

In [30]:
allocation = [0, 1, 1, 2]

In [31]:
replaceAlloc(allocation, 1, 2)

[0, 2, 1, 2]

In [37]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [112]:
def computeSAPproba(utilities, alloc, vehicle, tasks, ro = 0.1):
    Uv = np.array([utilities[tuple(replaceAlloc(allocation, vehicle, t))][vehicle] for t in tasks])
    #print("utilities for each task : "+str(Uv))
    proba = softmax(Uv/ro) # use softmax or logit function
    return proba

In [68]:
computeSAPproba(ut, allocation, 1, tasks, ro = 0.1) # very small ro value : probas are very unbalanced (no random)

utilities for each task : [2 5 4]


array([9.35719815e-14, 9.99954602e-01, 4.53978687e-05])

In [69]:
computeSAPproba(ut, allocation, 1, tasks, ro = 1) # small ro value : probas are unbalanced (very small random)

utilities for each task : [2 5 4]


array([0.03511903, 0.70538451, 0.25949646])

In [70]:
computeSAPproba(ut, allocation, 1, tasks, ro = 1000) # very high ro value : probas are ~ uniform (a lot of random)

utilities for each task : [2 5 4]


array([0.33277798, 0.33377781, 0.3334442 ])

In [113]:
def SpatialAdaptativePlay(utilities, vehicles, tasks, maxsteps, ro):
    k = 0
    cur_alloc = np.random.randint(0,len(tasks), len(vehicles))
    while k < maxsteps:
        v = np.random.choice(vehicles) # select uniformly the vehicle which is given the chance to update its task
        proba = computeSAPproba(ut, allocation, v, tasks, ro = 1) # compute proba to propose each task
        t = np.random.choice(tasks, p=proba) # choose the task to propose
        cur_alloc = replaceAlloc(cur_alloc, v, t) # update the allocation
        k += 1
    return list(cur_alloc)

In [132]:
# spatial adaptative play
SpatialAdaptativePlay(ut, vehicles, tasks, 1000, ro=1)

[0, 2, 0, 2]

### Comparing the different negociation methods :

In [140]:
# compare utility sum and execution time of the 3 methods
score_DMR = []
time_DMR = []
score_FP = []
time_FP = []
score_RM = []
time_RM = []
score_SAP = []
time_SAP = []
for step in range(10):# 10 different utility matrix
    ut = np.random.randint(0,10,shapes) # create utility matrix with random utilities
    for repeat in range(10): # 10 repeats on each utility matrix
        # best response dynamic
        start = time.time()
        alloc = tuple(bestResponseDynamic(ut, vehicles, tasks, 1000))
        duration = time.time() - start
        print("DMR : "+str(alloc))
        #print(ut[alloc])
        score_DMR.append(np.sum(ut[alloc]))
        time_DMR.append(duration)

        # fictitious play
        start = time.time()
        alloc = tuple(fictitiousPlay(ut, vehicles, tasks, 1000))
        duration = time.time() - start
        print("FP : "+str(alloc))
        #print(ut[alloc])
        score_FP.append(np.sum(ut[alloc]))
        time_FP.append(duration)

        # regret matching
        start = time.time()
        alloc = tuple(RegretMatching(ut, vehicles, tasks, 1000))
        duration = time.time() - start
        print("RM : "+str(alloc))
        #print(ut[alloc])
        score_RM.append(np.sum(ut[alloc]))
        time_RM.append(duration)

        # spatial adaptative play
        start = time.time()
        alloc = tuple(SpatialAdaptativePlay(ut, vehicles, tasks, 1000, ro=1))
        duration = time.time() - start
        print("SAP : "+str(alloc))
        #print(ut[alloc])
        score_SAP.append(np.sum(ut[alloc]))
        time_SAP.append(duration)

DMR : (1, 1, 1, 0)
FP : (1, 1, 1, 0)
RM : (2, 0, 0, 1)
SAP : (0, 1, 0, 0)
DMR : (1, 1, 1, 0)
FP : (1, 1, 1, 0)
RM : (1, 1, 1, 0)
SAP : (2, 2, 0, 0)
Execution stopped : maximum step overflowed, no EN found.
DMR : (1, 1, 2, 2)
FP : (1, 1, 1, 0)
RM : (2, 0, 0, 1)
SAP : (0, 1, 0, 0)
DMR : (1, 1, 1, 0)
FP : (1, 1, 1, 0)
RM : (1, 1, 1, 0)
SAP : (0, 1, 0, 0)
DMR : (1, 1, 1, 0)
FP : (1, 1, 1, 0)
RM : (1, 1, 1, 0)
SAP : (2, 1, 0, 0)
DMR : (1, 1, 1, 0)
FP : (1, 1, 1, 0)
RM : (1, 1, 1, 0)
SAP : (2, 1, 0, 0)
DMR : (1, 1, 1, 0)
FP : (1, 1, 1, 0)
RM : (2, 1, 2, 0)
SAP : (2, 1, 1, 0)
DMR : (1, 1, 1, 0)
FP : (1, 1, 1, 0)
RM : (1, 1, 1, 0)
SAP : (0, 1, 0, 0)
DMR : (1, 1, 1, 0)
FP : (1, 1, 1, 0)
RM : (2, 1, 2, 0)
SAP : (0, 1, 0, 0)
DMR : (1, 1, 1, 0)
FP : (1, 1, 1, 0)
RM : (2, 0, 0, 2)
SAP : (0, 1, 0, 2)
Execution stopped : maximum step overflowed, no EN found.
DMR : (0, 2, 0, 2)
FP : (1, 0, 1, 2)
RM : (1, 0, 1, 1)
SAP : (0, 0, 2, 2)
Execution stopped : maximum step overflowed, no EN found.
DMR : (1, 1,

FP : (2, 1, 0, 0)
RM : (0, 1, 0, 1)
SAP : (2, 1, 0, 0)
Execution stopped : maximum step overflowed, no EN found.
DMR : (1, 1, 2, 2)
FP : (2, 1, 0, 0)
RM : (0, 2, 0, 0)
SAP : (1, 1, 0, 0)
DMR : (0, 2, 0, 0)
FP : (2, 1, 0, 0)
RM : (0, 1, 2, 1)
SAP : (0, 1, 0, 1)
Execution stopped : maximum step overflowed, no EN found.
DMR : (0, 1, 2, 1)
FP : (2, 1, 0, 0)
RM : (2, 1, 0, 0)
SAP : (0, 1, 1, 0)
DMR : (0, 2, 0, 0)
FP : (2, 1, 0, 0)
RM : (2, 1, 2, 0)
SAP : (0, 1, 0, 0)
Execution stopped : maximum step overflowed, no EN found.
DMR : (1, 1, 0, 1)
FP : (2, 1, 0, 0)
RM : (2, 1, 0, 0)
SAP : (0, 1, 0, 0)
Execution stopped : maximum step overflowed, no EN found.
DMR : (1, 2, 2, 2)
FP : (2, 1, 0, 0)
RM : (0, 2, 1, 0)
SAP : (0, 1, 0, 1)
Execution stopped : maximum step overflowed, no EN found.
DMR : (0, 2, 2, 2)
FP : (2, 1, 0, 0)
RM : (0, 1, 2, 2)
SAP : (2, 1, 0, 1)
Execution stopped : maximum step overflowed, no EN found.
DMR : (0, 1, 2, 2)
FP : (2, 1, 0, 0)
RM : (0, 2, 1, 0)
SAP : (0, 1, 0, 1)
Execu

In [141]:
# on a first utility matrix :
print("mean DMR : "+str(np.mean(score_DMR))+" average time : "+str(np.mean(time_DMR)))
print("mean FP : "+str(np.mean(score_FP))+" average time : "+str(np.mean(time_FP)))
print("mean RM : "+str(np.mean(score_RM))+" average time : "+str(np.mean(time_RM)))
print("mean SAP : "+str(np.mean(score_SAP))+" average time : "+str(np.mean(time_SAP)))

mean DMR : 22.16 average time : 0.02802384853363037
mean FP : 23.3 average time : 11.998116133213044
mean RM : 23.94 average time : 0.6288598060607911
mean SAP : 18.68 average time : 0.1347698974609375
